# 샘플 데이터 생성 6개월   
연습에 사용할 샘플 데이터를 생성합니다.  
- 채널/시간/주중말 인벤 : table-chn, 200 * 24 * 2.  
- 셋탑/지역 인벤 : table-set, 1,000 만. rate, val, req 컬럼 6개월 분.  
- 셋탑/타겟 매핑 정보 : table-map  
- 타겟 인벤 : table-seg, 200.    
  
inventory db 에 생성.  
연산 전에 hive 에서 생성해서 넘겨주는 데이터  

*** 1,000만 6개월 컬럼 샘플 데이터 ***  
100 part  
- 데이터 생성 : 31 초 
- 파키 쓰기 :  68 초  


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = f'spark-01-sample-creation-6m-{scale}'
DB_NAME = 'inven'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '2g')\
    .config('spark.num.executors', '2')\
    .config('spark.executor.cores', '4').config('spark.executor.memory', '4g').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [13]:
scale = 1000

In [14]:
# 샘플 데이터 생성 
def create_setops():
    setop_count = scale * 10000
    inv_rate = 1.0 
    inv_val = 1000
    inv_req = 0
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E', 'ST_F', 'ST_G', 'ST_H', 'ST_I', 'ST_J']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setop_id = f'{s}_{i:07d}'
            setops.append([setop_id, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req\
                           , inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req, inv_rate,inv_val,inv_req])
            
    print(setops[-2:])
    return setops
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("inv_rate_01", FloatType())
        , StructField("inv_val_01", LongType())
        , StructField("inv_req_01", LongType())
        , StructField("inv_rate_02", FloatType())
        , StructField("inv_val_02", LongType())
        , StructField("inv_req_02", LongType())
        , StructField("inv_rate_03", FloatType())
        , StructField("inv_val_03", LongType())
        , StructField("inv_req_03", LongType())
        , StructField("inv_rate_04", FloatType())
        , StructField("inv_val_04", LongType())
        , StructField("inv_req_04", LongType())
        , StructField("inv_rate_05", FloatType())
        , StructField("inv_val_05", LongType())
        , StructField("inv_req_05", LongType())        
        , StructField("inv_rate_06", FloatType())
        , StructField("inv_val_06", LongType())
        , StructField("inv_req_06", LongType())        
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [15]:
%%time 
# 샘플 데이터 생성 및 확인 
sample_data = create_setops()
sample_schema = define_schema()
partition_num = 100
rdd = spark.sparkContext.parallelize(sample_data, partition_num)
df = spark.createDataFrame(rdd, sample_schema)
#df.show(10)

[['ST_J_0999998', 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0], ['ST_J_0999999', 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0, 1.0, 1000, 0]]
CPU times: user 29.3 s, sys: 3.08 s, total: 32.4 s
Wall time: 34 s


In [16]:
%%time 
# HDFS 에 /user/root/inven/setop 폴더와 파일이 생성 됨.  
tbl_setop_name = f'inven/table-set-6m-{scale}'
file_format = 'parquet'
df.write.save(path=tbl_setop_name, format=file_format, mode='append', sep=',')

CPU times: user 40.6 ms, sys: 8.08 ms, total: 48.7 ms
Wall time: 2min 1s


In [ ]:
%%time 
# 데이터가 큰 경우, 한번에 쓰기 보다, 나누어서 append로 쓰는 게 더 빠를 듯...
# 1,000 만 건을 순서대로 소팅해서 10번으로 나누어 써 본다. 

In [17]:
%%time 
# 기록한 파일 다시 읽어 들이기 
# 저장 결과 확인하기 
lines = spark.read.format(file_format).option('path', tbl_setop_name).load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 10,000,000
+------------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|       setop|inv_rate_01|inv_val_01|inv_req_01|inv_rate_02|inv_val_02|inv_req_02|inv_rate_03|inv_val_03|inv_req_03|inv_rate_04|inv_val_04|inv_req_04|inv_rate_05|inv_val_05|inv_req_05|inv_rate_06|inv_val_06|inv_req_06|
+------------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|ST_B_0899520|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      1000|         0|
|ST_B_0899521|        1.0|      1000|         0|        1.0|      1000|         0|        1.0|      

In [ ]:
# 타겟 : 세톱 매핑 정보 생성  


In [ ]:
# 샘플 타겟 정보 생성
def create_target():
    segs = []
    seg_count = 200
        for i in range(0, seg_count):
            seg_id = f'CATEGORY_{i:03d}'
            segs.append(seg_id)
            
    print(segs[-2:])
    return segs

In [ ]:
# 세톱 중 일부를 조회해서 타겟과 결합한다.  
# 타겟, 세톱 
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')

In [22]:
spark.stop()